In [2]:
import numpy as np

In [4]:
svddata = np.array([[1,1],[7,7]])
U,Sigma,VT = np.linalg.svd(svddata)
print(U,'\n',Sigma,'\n',VT)

[[-0.14142136 -0.98994949]
 [-0.98994949  0.14142136]] 
 [10.  0.] 
 [[-0.70710678 -0.70710678]
 [-0.70710678  0.70710678]]


In [15]:
def loaddata():
    return [[1,1,1,0,0],[2,2,2,0,0],[1,1,1,0,0],[5,5,5,0,0],[1,1,0,2,2],[0,0,0,3,3],[0,0,0,1,1]]

In [18]:
data = loaddata()
U,Sigma,VT = np.linalg.svd(data)
Sig3 = np.array([[Sigma[0],0,0],[0,Sigma[1],0],[0,0,Sigma[2]]])
np.dot(np.dot(U[:,:3],Sig3),VT[:3,:])
#可以看到返回的数据几乎与原数据一样

array([[ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
        -3.06102944e-16, -3.24716158e-16],
       [ 2.00000000e+00,  2.00000000e+00,  2.00000000e+00,
         1.56507791e-16,  1.19281363e-16],
       [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         1.59285357e-15,  1.57424036e-15],
       [ 5.00000000e+00,  5.00000000e+00,  5.00000000e+00,
        -1.16268486e-17, -1.04909757e-16],
       [ 1.00000000e+00,  1.00000000e+00, -1.08845596e-15,
         2.00000000e+00,  2.00000000e+00],
       [ 2.54588945e-16,  1.60102837e-15, -1.68123482e-15,
         3.00000000e+00,  3.00000000e+00],
       [ 1.30086946e-16,  5.78900087e-16, -6.70962949e-16,
         1.00000000e+00,  1.00000000e+00]])

In [22]:
def eclud(ina,inb):
    return 1.0/(1.0+np.linalg.norm(ina-inb))#np.linalg.norm 求范数
def pears(ina,inb):
    if len(ina)<3: return 1.0
    return 0.5+0.5*np.corrcoef(ina,inb,rowvar=0)[0][1]#第0组数组和第一组数组的相关系数
def coss(ina,inb):
    num = np.dot(ina.T,inb).astype(float)
    denom = np.linalg.norm(ina)*np.linalg.norm(inb)
    return 0.5+0.5*(num/denom)

In [50]:
arr = np.array(data2)
eclud(arr[:,0],arr[:,0])
eclud(arr[:,0],arr[:,4])

0.12973190755680383

In [52]:
coss(arr[:,0],arr[:,4])
#coss(arr[:,0],arr[:,0])

0.5

In [54]:
pears(arr[:,0],arr[:,4])
#pears(arr[:,0],arr[:,0])

0.20596538173840329

In [27]:
data

[[1, 1, 1, 0, 0],
 [2, 2, 2, 0, 0],
 [1, 1, 1, 0, 0],
 [5, 5, 5, 0, 0],
 [1, 1, 0, 2, 2],
 [0, 0, 0, 3, 3],
 [0, 0, 0, 1, 1]]

In [38]:
data2 = [[0, 0, 0, 2, 2],
           [0, 0, 0, 3, 3],
           [0, 0, 0, 1, 1],
           [1, 1, 1, 0, 0],
           [2, 2, 2, 0, 0],
           [5, 5, 5, 0, 0],
           [1, 1, 1, 0, 0]]

In [33]:
def standest(data,user,simeas,item):#user给定
    n = data.shape[1]
    simtal = 0.0
    ratsimtal = 0.0
    for j in range(n):
        userrating = data[user,j]
        if userrating == 0: continue
        overlap = np.nonzero(np.logical_and(data[:,item],data[:,j]))[0]#获取两个物品都非0的索引
        if len(overlap)==0:
            similar = 0
        else: similar = simeas(data[overlap,item],data[overlap,j])#计算j和item的物品都非0的相似度（默认余弦相似度）
        print('the %d and %d similarity is: %f' %(item,j,similar))
        simtal += similar
        ratsimtal += similar*userrating#每一个特征的值乘相关系数
    if simtal ==0: return 0
    else: return ratsimtal/simtal        

In [45]:
def recommend(data,user,N=3,simeas=coss,estmethod=standest):
    #print(np.nonzero(data[user,:]==0))
    unrated = np.nonzero(data[user,:]==0)[0]# 获取列索引
    if len(unrated)==0: return 'you rated everything'
    itemscores = []
    for item in unrated:
        estscore = estmethod(data,user,simeas,item)
        itemscores.append((item,estscore))
    return sorted(itemscores,key=lambda jj:jj[1],reverse=True)[:N]

In [39]:
data3 = np.array(data2)
data3[0,1]=data3[0,0]=data3[1,0]=data3[2,0]=4
data3[3,3]=2
data3

array([[4, 4, 0, 2, 2],
       [4, 0, 0, 3, 3],
       [4, 0, 0, 1, 1],
       [1, 1, 1, 2, 0],
       [2, 2, 2, 0, 0],
       [5, 5, 5, 0, 0],
       [1, 1, 1, 0, 0]])

In [57]:
recommend(data3,2,simeas=coss)

(array([1, 2]),)
the 1 and 0 similarity is: 1.000000
the 1 and 3 similarity is: 0.928746
the 1 and 4 similarity is: 1.000000
the 2 and 0 similarity is: 1.000000
the 2 and 3 similarity is: 1.000000
the 2 and 4 similarity is: 0.000000


[(2, 2.5), (1, 2.0243290220056256)]

In [31]:
a = np.logical_and(True,False)
b = np.logical_and([1,2],[0,1])
c = np.arange(5)
d = np.logical_and(c>1,c<4)
print(a,b,d)

False [False  True] [False False  True  True False]


In [59]:
def loaddata2():
    return[[0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 5],
           [0, 0, 0, 3, 0, 4, 0, 0, 0, 0, 3],
           [0, 0, 0, 0, 4, 0, 0, 1, 0, 4, 0],
           [3, 3, 4, 0, 0, 0, 0, 2, 2, 0, 0],
           [5, 4, 5, 0, 0, 0, 0, 5, 5, 0, 0],
           [0, 0, 0, 0, 5, 0, 1, 0, 0, 5, 0],
           [4, 3, 4, 0, 0, 0, 0, 5, 5, 0, 1],
           [0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 4],
           [0, 0, 0, 2, 0, 2, 5, 0, 0, 1, 2],
           [0, 0, 0, 0, 5, 0, 0, 0, 0, 4, 0],
           [1, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0]]

In [78]:
data4 = np.array(loaddata2())
U,Sigma,VT = np.linalg.svd(np.array(data4))
print(Sigma)
Sigma2 = Sigma**2
sums = np.sum(Sigma2)
print(np.sum(Sigma2[:3]),0.9*sums)

[15.77075346 11.40670395 11.03044558  4.84639758  3.09292055  2.58097379
  1.00413543  0.72817072  0.43800353  0.22082113  0.07367823]
500.5002891275791 487.7999999999994


In [75]:
#基于SVD评分设计
def svdest(data,user,simeas,item):
    n = data.shape[1]
    simtal = 0.0
    ratsimtal = 0.0
    U,Sigma,VT = np.linalg.svd(data)
    sig4 = np.eye(4)*Sigma[:4]
    formitem = np.dot(np.dot(data.T,U[:,:4]),np.linalg.inv(sig4))
    print(formitem)
    for j in range(n):
        userrating = data[user,j]
        if userrating ==0 or j ==item:
            continue
        similar = simeas(formitem[item,:].T,formitem[j,:].T)
        print('the %d and %d similarity is: %f' %(item,j,similar))
        simtal += similar
        ratsimtal += similar*userrating
    if simtal ==0: return 0
    else: return ratsimtal/simtal

In [76]:
recommend(data4,1,estmethod=svdest,simeas=pears)

(array([0, 1, 2, 4, 6, 7, 8, 9]),)
[[-0.45137416  0.03084799 -0.00290108  0.01189185]
 [-0.36239706  0.02584428 -0.00189127  0.01348796]
 [-0.46879252  0.03296133 -0.00281253  0.01656192]
 [-0.01007685 -0.34024331 -0.22728592  0.14546051]
 [-0.01567036 -0.38750193  0.61197998 -0.17137451]
 [-0.01664563 -0.52000097 -0.3608907  -0.14984063]
 [-0.00474684 -0.18887149 -0.00924222  0.94228361]
 [-0.46712774  0.00389831  0.03349951 -0.02080674]
 [-0.47223188  0.02853952 -0.00504059  0.00160266]
 [-0.01591788 -0.39205093  0.55707516  0.04356321]
 [-0.0552444  -0.52034959 -0.36330956 -0.19023805]]
the 0 and 3 similarity is: 0.341942
the 0 and 5 similarity is: 0.124132
the 0 and 10 similarity is: 0.116698
[[-0.45137416  0.03084799 -0.00290108  0.01189185]
 [-0.36239706  0.02584428 -0.00189127  0.01348796]
 [-0.46879252  0.03296133 -0.00281253  0.01656192]
 [-0.01007685 -0.34024331 -0.22728592  0.14546051]
 [-0.01567036 -0.38750193  0.61197998 -0.17137451]
 [-0.01664563 -0.52000097 -0.3608907  -

[(4, 3.3469521867021728), (9, 3.33537965732747), (6, 3.3071930278130375)]

In [77]:
data4.shape

(11, 11)

In [79]:
filename = '/Users/enjlife/machine-learning/machinelearninginaction/ch14/0_5.txt'

In [90]:
#示例：基于SVD的图像压缩
def printmat(inmat,thresh=0.8):
    for i in range(32):
        for j in range(32):
            if inmat[i,j]>thresh:
                print(1,end='')
            else: print(0,end='')
        print('')
def imgcompress(numsv=3,thresh=0.8):
    my1=[]
    
    for line in open(filename).readlines():
        newrow=[]
        for i in range(32):
            newrow.append(int(line[i]))
        my1.append(newrow)
    mymat = np.array(my1)
    print('**********original array*************')
    print(mymat.shape)
    printmat(mymat,thresh)
    U,Sigma,VT = np.linalg.svd(mymat)
    sigrecon = np.zeros((numsv,numsv))
    for k in range(numsv):
        sigrecon[k,k] = Sigma[k]
    reconmat = np.dot(np.dot(U[:,:numsv],sigrecon),VT[:numsv,:])
    print('**********reconstructed array using %d singular values*************'% numsv)
    print(reconmat.shape)
    printmat(reconmat,thresh)
    

可以看到，只需两个奇异值就能精确地对图像实现重构，U和V.T都是32x2的矩阵，有两个奇异值，因此总数字数据为64+64+2=130，与原数字相比，几乎压缩了10倍。

In [91]:
imgcompress(2)

**********original array*************
(32, 32)
00000000000000110000000000000000
00000000000011111100000000000000
00000000000111111110000000000000
00000000001111111111000000000000
00000000111111111111100000000000
00000001111111111111110000000000
00000000111111111111111000000000
00000000111111100001111100000000
00000001111111000001111100000000
00000011111100000000111100000000
00000011111100000000111110000000
00000011111100000000011110000000
00000011111100000000011110000000
00000001111110000000001111000000
00000011111110000000001111000000
00000011111100000000001111000000
00000001111100000000001111000000
00000011111100000000001111000000
00000001111100000000001111000000
00000001111100000000011111000000
00000000111110000000001111100000
00000000111110000000001111100000
00000000111110000000001111100000
00000000111110000000011111000000
00000000111110000000111111000000
00000000111111000001111110000000
00000000011111111111111110000000
00000000001111111111111110000000
00000000001111111111111110000